# 設置環境並載入套件

In [20]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/CorpusforDDL_compute/corpus_compute')
print(os.getcwd())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/CorpusforDDL_compute/corpus_compute


In [21]:
import pandas as pd
import re
import unicodedata
import pickle
from collections import Counter

In [22]:
# 載入事先寫好的 functions
from compute_parameters import *

In [23]:
# %%capture
# pip install -U ckip-transformers

In [24]:
# import ckip_transformers
# from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger
# ws_driver = CkipWordSegmenter(device=0)
# pos_driver = CkipPosTagger(device=0)

# 讀入語料

In [25]:
# 讀入國語日報新聞 txt 檔
mandarin_daily_path = '../CorpusforDDL/國語日報新聞/MandarinDailyNews-20181217'
all_files = os.listdir(mandarin_daily_path)

mandarin_daily_corpus = []
for f in all_files:
  with open(f'{mandarin_daily_path}/{f}') as f:
      lines = f.readlines()
      mandarin_daily_corpus.append(lines)

In [55]:
# 前處理
def preprocess_mandarin_daily(string):

  clean_string = re.sub('[\n△◎]', '', string) # 移除換行符號和△◎符號
  clean_string = re.sub("【\w+】", "", clean_string) # 移除【記者xxx報導】
  split_1 = re.split(r'(?<=。」|！」|？」)', clean_string) # 先用 。」 ！」 ？」 分隔
  split_2 = [re.split(r'(?<=[。！？])(?!」)', x) for x in split_1] # 再用 。！？ 分隔
  split = [item for sublist in split_2 for item in sublist] # 將 list of list 攤平
  split = [x for x in split if len(x)>1] # 濾掉空字串和只有一個標點符號的字串

  return split

In [56]:
mandarin_daily_corpus_preprocessed = []

for text in mandarin_daily_corpus:
  text_preprocessed = []
  for string in text:
    check_punctuation = re.search(r'[，。！？]', string) # 檢查字串是否包含，。！？
    check_chinese = re.search(r'[\u4e00-\u9fff]+', string) # 檢查字串是否包含中文字

    if check_punctuation and check_chinese:
      preprocessed = preprocess_mandarin_daily(string)
      text_preprocessed.append(preprocessed)

  mandarin_daily_corpus_preprocessed.append(text_preprocessed)

In [67]:
mandarin_daily_test = [item for sublist in mandarin_daily_corpus_preprocessed[0] for item in sublist]
mandarin_daily_test[0]

'今天是中華民國九十年元旦，也是二十一世紀的第一天，陳水扁總統發表跨世紀談話，並且向全民祝賀新年。'

# `ckip-transformers` 斷詞 + pos tag

In [28]:
# 斷詞
# mandarin_daily_ws  = ws_driver(mandarin_daily_test)
# with open('mandarin_daily_ws.pkl', 'wb') as f:
#     pickle.dump(mandarin_daily_ws, f)

In [ ]:
# pos tag
# mandarin_daily_pos = pos_driver(mandarin_daily_ws[10000:20000])
# with open('mandarin_daily_pos_10000.pkl', 'wb') as f:
#     pickle.dump(mandarin_daily_pos, f)

In [29]:
with open('../pkl_files/mandarin_daily_ws.pkl', 'rb') as f:
  mandarin_daily_ws = pickle.load(f)
mandarin_daily_ws_10000 = mandarin_daily_ws[:10000]

with open('../pkl_files/mandarin_daily_pos_10000.pkl', 'rb') as f:
  mandarin_daily_pos_10000 = pickle.load(f)

## 語料格式準備 - 1
斷詞 + pos tag

In [30]:
mandarin_daily_ws_pos = []

for sent_ws, sent_pos in zip(mandarin_daily_ws_10000, mandarin_daily_pos_10000):
  sent_ws_pos = []
  for ws, pos in zip(sent_ws, sent_pos):
    ws_pos = f"{ws}({pos})"
    sent_ws_pos.append(ws_pos)
  mandarin_daily_ws_pos.append(' '.join(sent_ws_pos))

In [31]:
mandarin_daily_ws_pos[0]

'今天(Nd) 是(SHI) 中華民國(Nc) 九十年(Nd) 元旦(Nd) ，(COMMACATEGORY) 也(D) 是(SHI) 二十一世紀(Nd) 的(DE) 第一(Neu) 天(Nf) ，(COMMACATEGORY) 陳水扁(Nb) 總統(Na) 發表(VC) 跨(VC) 世紀(Na) 談話(Na) ，(COMMACATEGORY) 並且(Cbb) 向(P) 全民(Na) 祝賀(VC) 新年(Nd) 。(PERIODCATEGORY)'

## 語料格式準備 - 2
斷好詞，無標點符號

In [32]:
mandarin_daily_ws = []

for sentence in mandarin_daily_ws_10000:
  clean_sent = []
  for word in sentence:
    if re.match(r'[A-z0-9\w]+', word):
      clean_sent.append(word)
  mandarin_daily_ws.append(clean_sent)

In [33]:
mandarin_daily_ws[0]

['今天',
 '是',
 '中華民國',
 '九十年',
 '元旦',
 '也',
 '是',
 '二十一世紀',
 '的',
 '第一',
 '天',
 '陳水扁',
 '總統',
 '發表',
 '跨',
 '世紀',
 '談話',
 '並且',
 '向',
 '全民',
 '祝賀',
 '新年']

## 語料格式準備 - 3
完整句子，含標點符號

In [39]:
mandarin_daily_text = []
for sent in mandarin_daily_ws_pos:
  res = re.sub(r'\([A-z0-9_]+\)', '', sent)
  res = res.replace(' ', '')
  mandarin_daily_text.append(res)

mandarin_daily_text[0]

'今天是中華民國九十年元旦，也是二十一世紀的第一天，陳水扁總統發表跨世紀談話，並且向全民祝賀新年。'

# 句子長度

In [40]:
sentence_length = [len(sentence) for sentence in mandarin_daily_ws]
sentence_length[:10]

[22, 34, 13, 43, 35, 38, 38, 38, 20, 60]

# 高頻詞與低頻詞

In [41]:
high_low_freq = [get_high_low_freq(sentence) for sentence in mandarin_daily_ws]
high_low_freq[:10]

[{'High': 22},
 {'High': 33, 'Low': 1},
 {'High': 13},
 {'High': 43},
 {'High': 35},
 {'High': 37, 'Low': 1},
 {'High': 38},
 {'High': 37, 'Low': 1},
 {'High': 20},
 {'High': 60}]

# 詞頻

In [42]:
word_freq = [get_word_freq(sentence) for sentence in mandarin_daily_ws]
word_freq[0]

'今天(73277) 是(4642407) 中華民國(5430) 九十年(105) 元旦(1180) 也(1590760) 是(4642407) 二十一世紀(3330) 的(16029838) 第一(172544) 天(330413) 陳水扁(1345) 總統(23662) 發表(25858) 跨(8128) 世紀(16204) 談話(16731) 並且(60996) 向(231672) 全民(1980) 祝賀(1833) 新年(2207)'

# 詞彙等級

In [43]:
word_level = [get_word_level(sentence) for sentence in mandarin_daily_ws]
word_level[0]

{'第1級': 6, '第5級': 4, 'Unknown': 4, '第4*級': 2, '第2級': 2, '第4級': 3, '第2*級': 1}

# 詞彙長度

In [44]:
long_word_count = [get_long_word_count(sentence) for sentence in mandarin_daily_ws]
long_word_count[:10]

[4, 1, 2, 1, 0, 1, 1, 2, 1, 2]

# 完整的句子

In [45]:
is_complete_sentence = [get_complete_sentence(sentence) for sentence in mandarin_daily_ws_pos]
is_complete_sentence[:10]

['Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y']

# 完整的語境

In [46]:
is_complete_context = [get_complete_context(x) for x in mandarin_daily_ws_pos]
is_complete_context[:10]

['Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'N', 'N', 'Y']

# Black list

In [47]:
is_blacklist = [get_blacklist(x) for x in mandarin_daily_text]
is_blacklist[:10]

['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N']

# Grey list

In [48]:
is_greylist = [get_greylist(x) for x in mandarin_daily_ws_pos]
is_greylist[:10] 

['Y', 'Y', 'Y', 'N', 'Y', 'N', 'Y', 'Y', 'Y', 'N']

# Make dataframe

In [49]:
mandarin_daily_df = pd.DataFrame({'sentence': mandarin_daily_text,
                        'sentence_preprocessed': [' '.join(sent) for sent in mandarin_daily_ws],
                        'sentence_length': sentence_length,
                        'word_freq': word_freq,
                        'high_low_freq': high_low_freq,
                        'word_level': word_level,
                        'long_word_count': long_word_count,
                        'is_complete_sentence': is_complete_sentence,
                        'is_complete_context': is_complete_context,
                        'is_blacklist': is_blacklist,
                        'is_greylist': is_greylist})

In [50]:
mandarin_daily_df = mandarin_daily_df[mandarin_daily_df['sentence_length']>0]
mandarin_daily_df

,sentence,sentence_preprocessed,sentence_length,word_freq,high_low_freq,word_level,long_word_count,is_complete_sentence,is_complete_context,is_blacklist,is_greylist
0,今天是中華民國九十年元旦，也是二十一世紀的第一天，陳水扁總統發表跨世紀談話，並且向全民祝賀新年。,今天 是 中華民國 九十年 元旦 也 是 二十一世紀 的 第一 天 陳水扁 總統 發表 跨 ...,22,今天(73277) 是(4642407) 中華民國(5430) 九十年(105) 元旦(11...,{'High': 22},"{'第1級': 6, '第5級': 4, 'Unknown': 4, '第4*級': 2, ...",4,Y,Y,N,Y
1,陳總統表示，過去政府的「戒急用忍」政策有當時的背景及其必要，但是未來政府將以「積極開放，有效...,陳 總統 表示 過去 政府 的 戒急用忍 政策 有 當時 的 背景 及 其 必要 但是 未來...,34,陳(10579) 總統(23662) 表示(74452) 過去(88758) 政府(9266...,"{'High': 33, 'Low': 1}","{'Unknown': 4, '第4級': 8, '第4*級': 5, '第1級': 8, ...",1,Y,Y,N,Y
2,陳總統指出，二十一世紀，經濟競爭力依然是我們賴以生存的命脈。,陳 總統 指出 二十一世紀 經濟 競爭力 依然 是 我們 賴以 生存 的 命脈,13,陳(10579) 總統(23662) 指出(38456) 二十一世紀(3330) 經濟(83...,{'High': 13},"{'Unknown': 4, '第4級': 2, '第6級': 1, '第1級': 3, '...",2,Y,N,N,Y
3,面對全球景氣循環的低潮以及包括美國、日本等國家經濟成長的向下修正，臺灣地區的經濟繁榮將再次面...,面對 全球 景氣 循環 的 低潮 以及 包括 美國 日本 等 國家 經濟 成長 的 向 下 ...,43,面對(62914) 全球(21104) 景氣(1794) 循環(11162) 的(16029...,{'High': 43},"{'第4*級': 3, '第4級': 6, '第5級': 8, '第1級': 12, '第6...",1,Y,Y,N,N
4,另外，因應我們即將加入世界貿易組織，陳總統說，政府有責任面對全新的經濟情勢，將包括兩岸經貿在...,另外 因應 我們 即將 加入 世界 貿易 組織 陳 總統 說 政府 有 責任 面對 全新 的...,35,另外(48233) 因應(6201) 我們(979260) 即將(16780) 加入(288...,{'High': 35},"{'第3級': 2, '第6級': 3, '第1級': 6, '第4級': 6, '第2級'...",0,Y,Y,N,Y
...,...,...,...,...,...,...,...,...,...,...,...
9995,鄭立娜老師表示，植物和動物的生態，老師們常常使用，教學更活潑了。,鄭立娜 老師 表示 植物 和 動物 的 生態 老師 們 常常 使用 教學 更 活潑 了,16,鄭立娜(Unknown) 老師(88888) 表示(74452) 植物(19121) 和(1...,"{'Low': 1, 'High': 15}","{'Unknown': 6, '第1級': 4, '第4*級': 1, '第5級': 1, ...",1,Y,Y,N,Y
9996,屬於小朋友的四月即將來臨，基隆市政府要在春假營造屬於小朋友的歡樂節慶，「二○○一基隆小丑節歡...,屬於 小朋友 的 四月 即將 來臨 基隆市 政府 要 在 春假 營造 屬於 小朋友 的 歡樂...,40,屬於(39478) 小朋友(9889) 的(16029838) 四月(14996) 即將(1...,"{'High': 37, 'Low': 3}","{'第5級': 7, '第2級': 4, '第1級': 4, 'Unknown': 12, ...",8,Y,Y,N,Y
9997,小丑節活動時間從三十一日到四月八日，在每天上午十點到晚上八點舉行，活動地點主要以碧沙漁港四周...,小丑節 活動 時間 從 三十一日 到 四月 八日 在 每 天 上午 十點 到 晚上 八點 舉...,66,小丑節(Unknown) 活動(85743) 時間(213595) 從(496546) 三十...,"{'Low': 6, 'High': 60}","{'Unknown': 25, '第2級': 3, '第1級': 10, '第1*級': 4...",7,Y,Y,N,Y
9998,春假期間小朋友想不想充電？,春假 期間 小朋友 想 不 想 充電,7,春假(158) 期間(32981) 小朋友(9889) 想(449884) 不(228252...,{'High': 7},"{'第3*級': 1, '第4*級': 1, '第2級': 1, '第1級': 3, '第4...",1,Y,N,N,N


In [51]:
mandarin_daily_df.to_csv('../results/mandarin_daily/mandarin_daily_parameters_example.csv', index = False)

## Concordance

In [52]:
from nltk.text import Text

In [53]:
target_words = ['難得', '畢竟', '的確', '難免', '總是', '有助於']
corpus = [item for sublist in mandarin_daily_ws for item in sublist]
text = Text(corpus)
dfs = []

for word in target_words:

  con_list = text.concordance_list(word)
  right_word = [x.right[0] for x in con_list]
  left_word = [x.left[-1] for x in con_list]
  context = [x.left + [word] + x.right for x in con_list]
  context = [' '.join(x) for x in context]

  df = pd.DataFrame({'left_word': left_word,
                     'target_word': word,
                     'right_word': right_word,
                     'context': context})
  dfs.append(df)

In [54]:
concordance_df = pd.concat(dfs)
concordance_df.to_csv('../results/mandarin_daily/mandarin_daily_concordance_df.csv', index = False)